In [513]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ISLP.models import ModelSpec as MS
import statsmodels.api as sm
import keras
from keras import layers
from keras import ops
from sklearn.model_selection import train_test_split

In [514]:
import os
PathToRepo = os.path.normpath(os.getcwd() + os.sep + os.pardir)
Df = pd.read_csv(PathToRepo + '\\Data\\Faellesdata_cleaned.csv')

x = Df.drop('IsMigratorInt', axis=1) 
for i in range(0, x.shape[1]): #Standardizing the data
    x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

x_train, x_test, y_train, y_test = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [515]:
x_train = MS(x_train, intercept = False).fit_transform(x_train)

In [517]:
modelnn = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [518]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=10)

Epoch 1/10


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8024 - loss: 0.4427
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8422 - loss: 0.3409
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8535 - loss: 0.3057
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8594 - loss: 0.2950
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8603 - loss: 0.2852
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8622 - loss: 0.2780
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8656 - loss: 0.2675
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8785 - loss: 0.2601
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8714 - loss: 0.2684
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8790 - loss: 0.2521


In [519]:
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8752 - loss: 0.2594  


[0.2658539116382599, 0.8679999709129333]

In [520]:
from sklearn.metrics import accuracy_score

In [527]:

Models = []
modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 1', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layers': i, 
                   'Neurons': neurons, 
                   'Activation': activation})

ModelsDf = pd.DataFrame(Models)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step


In [528]:
modelsoftmax = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [529]:
modelsoftmax.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelsoftmax.fit(x_train, keras.utils.to_categorical(y_train), epochs=10)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8277 - loss: 0.4221
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8459 - loss: 0.3291
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8576 - loss: 0.2949
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8624 - loss: 0.2825
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8617 - loss: 0.2886
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8686 - loss: 0.2680
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8693 - loss: 0.2627
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8702 - loss: 0.2621
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8628 - loss: 0.2662
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8823 - loss: 0.2535


In [530]:
Models = []
modelnn = modelsoftmax

modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 2'
                   , 'Optimizer': modelnn.optimizer.get_config()['name']
                   , 'Loss': modelnn.loss
                    , 'Accuracy': accuracy_score(y_test, np.argmax(modelsoftmax.predict(MS(x_test, intercept = False).fit_transform(x_test)), axis=1))
                   , 'Layers': i, 'Neurons': neurons
                   , 'Activation': activation})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layers', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step


In [533]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(1, activation='sigmoid')
])
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8633 - loss: 0.2728  


[0.27059683203697205, 0.8600000143051147]

In [534]:
Models = []
modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 3', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layers': i, 
                   'Neurons': neurons, 
                   'Activation': activation})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layers', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step


In [535]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [536]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8577 - loss: 0.2923


[0.2906586229801178, 0.8529999852180481]

In [537]:
Models = []
modelconfig = modelnn.get_config()
modelconfig['layers'][2]
for i in range(1, len(modelconfig['layers'])):
    layer_type = modelconfig['layers'][i]['class_name']
    if layer_type == 'Dropout':
        rate = modelconfig['layers'][i]['config']['rate']
        Models.append({'Model': 'Neural Network 4', 
                       'Optimizer': modelnn.optimizer.get_config()['name'], 
                       'Loss': modelnn.loss, 
                       'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                       'Layers': i, 
                       'Rate': rate})
        continue
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    Models.append({'Model': 'Neural Network 4', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layers': i, 
                   'Neurons': neurons, 
                   'Activation': activation})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layers', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 913us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


Note That the full data file is not included in the repository, as it is too large and that the code below takes over 30 mins to run. The code above just loads the model and calculates the accuracy score

In [538]:
# from astropy.io import ascii
# FullData = ascii.read(PathToRepo + '\\Data\\m12i_res7100_mhdcv.disk.ecsv', guess = False)
# FullData = FullData.to_pandas()

In [539]:
# FullData['IsMigrator'] = abs(FullData['Rcyl'] - FullData['Rcyl_form']) > 1.5
# FullData['IsMigratorInt'] = FullData['IsMigrator'].astype(int)
# FullData = FullData.drop(['rsph', 'x', 'y',  'vx', 'vy', 
#        'rsph_form', 'x_form', 'y_form', 'z_form', 'vx_form', 'vy_form',
#        'vz_form', 'Rcyl_form', 'phi_form', 'vRcyl_form', 'vphi_form', "IsMigrator"], axis = 1)

In [540]:
# Df = FullData
# x = Df.drop('IsMigratorInt', axis=1) 
# for i in range(0, x.shape[1]): #Standardizing the data
#     x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

# x_train, x_test, y_train, y_test = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [541]:
# modelnn = keras.Sequential([
#     layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
#     layers.Dense(1, activation='sigmoid')
# ])
# modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# modelnn.fit(x_train, y_train, epochs=10)

In [543]:
Models = []
modelnn = keras.models.load_model('modelnn.keras')

modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 5', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layers': i, 
                   'Neurons': neurons, 
                   'Activation': activation})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layers', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step


In [544]:
asd = ModelsDf

In [545]:
ModelsDf = asd

ModelsDf
ModelsDf.index = ModelsDf['Model']
ModelsDf = ModelsDf.drop('Model', axis = 1)

In [546]:
ModelsDf.index = ModelsDf.index.rename('Model')

In [547]:
ModelsDf.index = pd.MultiIndex.from_tuples([(ModelsDf.index[i],i) for i in range(0, ModelsDf.shape[0])])

In [548]:
ModelsDf['Loss'].replace("_", "\\textunderscore ", regex=True, inplace=True)

C:\Users\lorentsen\AppData\Local\Temp\ipykernel_51116\2191194268.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ModelsDf['Loss'].replace("_", "\\textunderscore ", regex=True, inplace=True)


In [549]:
ModelsDf.to_latex(PathToRepo + '\\Tables\\Models.tex', index_names = True, na_rep='-', multirow = True, float_format="%.3f", caption="Differnet neural networks", label="tab:RawData", position="h!")
ModelsDf

Optimizer                                     Loss  \
Neural Network 1 0       adam       binary\textunderscore crossentropy   
                 1       adam       binary\textunderscore crossentropy   
                 2       adam       binary\textunderscore crossentropy   
Neural Network 2 3       adam  categorical\textunderscore crossentropy   
                 4       adam  categorical\textunderscore crossentropy   
                 5       adam  categorical\textunderscore crossentropy   
Neural Network 3 6       adam       binary\textunderscore crossentropy   
                 7       adam       binary\textunderscore crossentropy   
Neural Network 4 8       adam       binary\textunderscore crossentropy   
                 9       adam       binary\textunderscore crossentropy   
                 10      adam       binary\textunderscore crossentropy   
                 11      adam       binary\textunderscore crossentropy   
                 12      adam       binary\textunderscore crossentropy   
Neural Network 5 13   rmsprop       binary\textunderscore crossentropy   
                 14   rmsprop       binary\textunderscore crossentropy   

                     Accuracy  Layers  Neurons Activation  Rate  
Neural Network 1 0      0.868       1     64.0       relu   NaN  
                 1      0.868       2     64.0       relu   NaN  
                 2      0.868       3      1.0    sigmoid   NaN  
Neural Network 2 3      0.867       1     64.0       relu   NaN  
                 4      0.867       2     64.0       relu   NaN  
                 5      0.867       3      2.0    softmax   NaN  
Neural Network 3 6      0.860       1     16.0       relu   NaN  
                 7      0.860       2      1.0    sigmoid   NaN  
Neural Network 4 8      0.853       1     16.0       relu   NaN  
                 9      0.853       2      NaN        NaN   0.5  
                 10     0.853       3     16.0       relu   NaN  
                 11     0.853       4      NaN        NaN   0.5  
                 12     0.853       5      1.0    sigmoid   NaN  
Neural Network 5 13     0.880       1     16.0       relu   NaN  
                 14     0.880       2      1.0    sigmoid   NaN

In [ ]:
# modelnn.save('modelnn.keras')